# Importing the libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

#libraries for preprocessing
import numpy as np 
import pandas as pd 

#libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns 

#libraries for data based / driven models 
from statsmodels.tsa.seasonal import seasonal_decompose 
from statsmodels.tsa.holtwinters import SimpleExpSmoothing #for simple exponential smoothning
from statsmodels.tsa.holtwinters import Holt #for holts/Double exponential model
from statsmodels.tsa.holtwinters import ExponentialSmoothing 

#
#libraries for ploting ACF and PACF plots
import statsmodels.graphics.tsaplots as tsa_plots 

#libraries for checking the stationarity
from statsmodels.tsa.stattools import adfuller
#from statsmodels.tsa.statespace.tools import diff as diff

#libraries for ARMA model
from statsmodels.tsa.arima_model import ARMA

#libraries for model driven models (ARIMA)
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm


# Importing the data

In [2]:
gold_price = pd.read_csv('Gold_data.csv' , parse_dates=['date'])

FileNotFoundError: [Errno 2] No such file or directory: 'Gold_data.csv'

In [ ]:
gold_price

# Preprocessing the data 

In [ ]:
gold_price['date']=pd.to_datetime(gold_price['date'])

In [ ]:
gold_price.info()

In [ ]:
gold_price = gold_price.set_index('date' , drop=True)

In [ ]:
gold_price

In [ ]:
gold_price.info()

In [ ]:
gold_price.shape

# Checking for any missing dates in our data 

In [ ]:
print(pd.date_range(start='2016-01-01' , end='2021-12-21').difference(gold_price.index))

since from above output we get to know that there are no missing dates

# Checking for the null values 

In [ ]:
print('missing_price :',gold_price.isna().sum())

there are null values

# Checking for the outlayers

In [ ]:
fig , ax=plt.subplots(figsize=(12,6))
sns.boxplot(data=gold_price , x=gold_price.index.year , y='price' , ax=ax)

# Data Understanding 

# EDA==>Exploratory data analysis

In [ ]:
#day wise 
plt.figure(figsize=(14,7))
gold_price.price.plot()
plt.ylabel('Price')

In [ ]:
#day wise
#considering only one year
plt.figure(figsize=(14,7))
gold_price.price[:365].plot()
plt.ylabel('Price')

In [ ]:
#weekly
plt.figure(figsize=(14,7))
gold_price['price'].resample(rule='W').mean().plot()
plt.ylabel('price')
plt.xlabel('Based on weekly')

#month wise
plt.figure(figsize=(14,7))
gold_price["price"].resample(rule="MS").mean().plot()
plt.ylabel("Price")
plt.xlabel('based on monthly')

#Ouaterly wise
plt.figure(figsize=(14,7))
gold_price["price"].resample(rule="QS").mean().plot()
plt.ylabel("Price")
plt.xlabel('based on quaterly')

#anually wise
plt.figure(figsize=(14,7))
gold_price["price"].resample(rule="A").mean().plot()
plt.ylabel("Price")
plt.xlabel('Based on Anually')

In [ ]:
#Annually
sns.barplot(data=gold_price , x=gold_price.index.year, y='price')

# Time Series Decomposition plots

In [ ]:
decompose_ts_add = seasonal_decompose(gold_price['price'],extrapolate_trend='freq')
decompose_ts_add.plot()
plt.rcParams['figure.figsize'] = (12, 8)
plt.show()

In [ ]:
#for day ways 
#by considering only the one month data 

decompose_ts_add=seasonal_decompose(gold_price[:31])
decompose_ts_add.plot()
#plt.rcParams['figure.figsize'] = (12, 8)
plt.show()

In [ ]:
#for day wise 
#considering the one year data
decompose_ts_add=seasonal_decompose(gold_price[:365])
decompose_ts_add.plot()
plt.show()
plt.rcParams['figure.figsize'] = (12, 8)

from above decomposition plots we can say that 
our data is not following trend but there is a seasionality

# Distribution Plot

In [ ]:
sns.distplot(gold_price)

# Splitting the data into train and test

In [ ]:
#splitting the data
Train = gold_price.head(2039)
Test = gold_price.tail(143)

In [ ]:
Train

In [ ]:
Test

# Moving average

In [ ]:
plt.figure(figsize=(14,4))
gold_price.price.plot(label='org')
plt.title('Original Series')

for i in range (25,35):
    plt.figure(figsize=(14,4))
     
    gold_price['price'].rolling(i).mean().plot(label=str(i))
    plt.title('Moving Average : '+str(i))
    plt.legend(loc='best')
    plt.show()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Evaluation Metric MAPE

In [ ]:
def MAPE(pred,org):
    temp = np.abs((pred-org)/org)*100
    return np.mean(temp)

# Model Preparation for forecasting

# Data Driven forecasting 

# Model Based or Driven Forcasting technique

# Using the data driven models

# Simple Exponential Method 

In [ ]:
#checking for the best smoothing 

In [ ]:
import numpy as np
x=np.linspace(0.1,1,20)
for i in x :
    ses_model = SimpleExpSmoothing(Train["price"]).fit(smoothing_level=i)
    pred_ses = ses_model.predict(start = Test.index[0],end = Test.index[-1])
    score=MAPE(pred_ses,Test.price) 
    print("smoothing_level ",i,"-->","MAPE Error",score)


In [ ]:
ses_model = SimpleExpSmoothing(Train['price']).fit(smoothing_level=0.2)
pred_ses = ses_model.predict(start=Test.index[0] , end=Test.index[-1])
simple_exp_model=MAPE(pred_ses , Test.price)
simple_exp_model

# Double or holt exponential method

In [ ]:
import numpy as np
alphas=np.arange(0.1,1,0.9/10)
betas=np.arange(0.1,1,0.9/10)
l=[]
for i in alphas :
    for j in betas:
        hw_model = Holt(Train["price"]).fit(smoothing_level=i, smoothing_slope=j)
        pred_hw = hw_model.predict(start = Test.index[0],end = Test.index[-1])
        score=MAPE(pred_hw,Test.price)
        print("smoothing_level ",i," smoothing_slope",j,"-->","MAPE Error",score)
        l.append((i,j,score))
        
        

In [ ]:
hw_model = Holt(Train['price']).fit(smoothing_level=0.5,smoothing_slope =0.5)
pred_hw = hw_model.predict(start=Test.index[0],end=Test.index[-1])
double_exp_model=MAPE(pred_hw,Test.price)
double_exp_model

# triple  or Holt Winner Exponential Method

In [ ]:
#seasonal =additive ,trend=additive

In [ ]:
hwe_model_add_add = ExponentialSmoothing(Train['price'],seasonal='add',trend='add',seasonal_periods=31).fit()
pred_hwe_add_add = hwe_model_add_add.predict(start=Test.index[0],end=Test.index[-1])
triple_exp_add_add=MAPE(pred_hwe_add_add,Test.price)
triple_exp_add_add

In [ ]:
fc=hwe_model_add_add.forecast(143)
# Make as pandas series
fc_series = pd.Series(fc, index=Test.index)
#lower_series = pd.Series(conf[:, 0], index=test.index)
#upper_series = pd.Series(conf[:, 1], index=test.index)


# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(Train, label='training')
plt.plot(Test, label='actual')
plt.plot(fc_series, label='forecast')
#plt.fill_between(lower_series.index, upper_series, 
                 #color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
fc=hwe_model_add_add.forecast(143)
# Make as pandas series
fc_series = pd.Series(fc, index=Test.index)
#lower_series = pd.Series(conf[:, 0], index=test.index)
#upper_series = pd.Series(conf[:, 1], index=test.index)


# Plot
plt.figure(figsize=(12,5), dpi=100)
#plt.plot(Train, label='training')
plt.plot(Test, label='actual')
plt.plot(fc_series, label='forecast')
#plt.fill_between(lower_series.index, upper_series, 
                 #color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
#seasonal=multiplicative , trend=additive

In [ ]:
hwe_model_mul_add = ExponentialSmoothing(Train['price'],seasonal='mul',trend='add',seasonal_periods=31).fit()
pred_hwe_mul_add= hwe_model_mul_add.predict(start=Test.index[0],end=Test.index[-1])
triple_exp_mul_add=MAPE(pred_hwe_mul_add,Test.price)
triple_exp_mul_add

In [ ]:
fc=hwe_model_mul_add.forecast(143)
# Make as pandas series
fc_series = pd.Series(fc, index=Test.index)
#lower_series = pd.Series(conf[:, 0], index=test.index)
#upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(Train, label='training')
plt.plot(Test, label='testing')
plt.plot(fc_series, label='forecast')
#plt.fill_between(lower_series.index, upper_series, 
                 #color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
fc=hwe_model_mul_add.forecast(143)
# Make as pandas series
fc_series = pd.Series(fc, index=Test.index)
#lower_series = pd.Series(conf[:, 0], index=test.index)
#upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
#plt.plot(Train, label='training')
plt.plot(Test, label='testing')
plt.plot(fc_series, label='forecast')
#plt.fill_between(lower_series.index, upper_series, 
                 #color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
#seasonal = additive , trend = multiplicative

In [ ]:
hwe_model_add_mul = ExponentialSmoothing(Train['price'],seasonal='add',trend='mul',seasonal_periods=31).fit()
pred_hwe_add_mul = hwe_model_add_mul.predict(start=Test.index[0],end=Test.index[-1])
triple_exp_add_mul=MAPE(pred_hwe_add_mul,Test.price)
triple_exp_add_mul

In [ ]:
fc=hwe_model_add_mul.forecast(143)
# Make as pandas series
fc_series = pd.Series(fc, index=Test.index)
#lower_series = pd.Series(conf[:, 0], index=test.index)
#upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(Train, label='training')
plt.plot(Test, label='actual')
plt.plot(fc_series, label='forecast')
#plt.fill_between(lower_series.index, upper_series, 
                 #color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
fc=hwe_model_add_mul.forecast(143)
# Make as pandas series
fc_series = pd.Series(fc, index=Test.index)
#lower_series = pd.Series(conf[:, 0], index=test.index)
#upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
#plt.plot(Train, label='training')
plt.plot(Test, label='actual')
plt.plot(fc_series, label='forecast')
#plt.fill_between(lower_series.index, upper_series, 
                 #color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
#seasonal = multiplicative , trend = multiplicative

In [ ]:
hwe_model_mul_mul = ExponentialSmoothing(Train['price'],seasonal='mul',trend='mul',seasonal_periods=31).fit()
pred_hwe_mul_mul = hwe_model_mul_mul.predict(start=Test.index[0],end=Test.index[-1])
triple_exp_mul_mul=MAPE(pred_hwe_mul_mul,Test.price)
triple_exp_mul_mul

In [ ]:
fc=hwe_model_mul_mul.forecast(143)
# Make as pandas series
fc_series = pd.Series(fc, index=Test.index)
#lower_series = pd.Series(conf[:, 0], index=test.index)
#upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(Train, label='training')
plt.plot(Test, label='actual')
plt.plot(fc_series, label='forecast')
#plt.fill_between(lower_series.index, upper_series, 
                 #color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
fc=hwe_model_mul_mul.forecast(143)
# Make as pandas series
fc_series = pd.Series(fc, index=Test.index)
#lower_series = pd.Series(conf[:, 0], index=test.index)
#upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
#plt.plot(Train, label='training')
plt.plot(Test, label='actual')
plt.plot(fc_series, label='forecast')
#plt.fill_between(lower_series.index, upper_series, 
                 #color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

# Model driven forcasting

# ARIMA and SARIMA


In [ ]:
gold_price

# Visualize the data

In [ ]:
gold_price.plot()

# Checking for the stationarity of the data

# checking the stationarity by using the ADFuller technique

In [ ]:
#gold_price = adfuller(gold_price['price'])

In [ ]:
#ADF Test nul hypothesis -non stationarity if p-value <5% reject null hypothesis
adf = adfuller(gold_price['price'])[1]
print(f'p value:{adf.round(4)}',',series is stationary' if adf<0.05 else ',series is non stationary')

# Differencing

In [ ]:
stationary_gold_price = gold_price.diff(1).dropna()
adf2 = adfuller(stationary_gold_price)[1]
print(f'p value:{adf2}',',series is stationary' if adf2<0.05 else ",series is non stationary")
stationary_gold_price.plot()

In [ ]:
#sns.distplot(stationary_gold_price)

# Checking the Distribution of the data after differencing

In [ ]:
sns.distplot(stationary_gold_price)

from the above distplot we got to know that our data is normally distributed

# checking the stationarity by seeing the mean , standard deviation 

# Differencing

 #differencing value d=1
plt.figure(figsize=(16,7))
fig = plt.figure(1)
gold_price_diff = gold_price.price - gold_price.price.shift(periods=1)
plt.plot(gold_price_diff)

#Determing rolling statistics 
rollingmean = gold_price_diff.rolling(7).mean()
rollingstd = gold_price_diff.rolling(7).std()

#plot rolling statistics :
orig = plt.plot(gold_price_diff,color='blue' , label='Original')
mean = plt.plot(rollingmean , color='red' , label='Rolling mean')
std = plt.plot(rollingstd , color='black',label = 'Rolling std')
plt.legend(loc='best')
plt.title('Rolling mean and rolling std after diff')
plt.show(block=False)

# Ploting the Auto-correlation plot

# ACF plots and PACF plots

In [ ]:
tsa_plots.plot_acf(gold_price.price)
tsa_plots.plot_pacf(gold_price.price)
plt.show()

# ACF and PACF plots for data after considering the differencing  

In [ ]:
tsa_plots.plot_acf(stationary_gold_price.price)
tsa_plots.plot_pacf(stationary_gold_price.price)
plt.show()

# Creating the ARMA Model

# creating the ARIMA model

In [ ]:
#for non seasonal data
#p=1,d=1,q=1

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

arima_model_train_1_1 = ARIMA(Train.price , order=(1,1,1)).fit()

pred_arima = arima_model_train_1_1.predict(start=Test.index[0],end=Test.index[-1])
ARIMA_MODEL_TRAIN_1_1=MAPE(pred_arima , Test.price)
ARIMA_MODEL_TRAIN_1_1

In [ ]:
fc=arima_model_train_1_1.forecast(143)
# Make as pandas series
fc_series = pd.Series(fc, index=Test.index)
#lower_series = pd.Series(conf[:, 0], index=test.index)
#upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(Train, label='training')
plt.plot(Test, label='actual')
plt.plot(fc_series, label='forecast')
#plt.fill_between(lower_series.index, upper_series, 
                 #color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
#for non seasonal data
#p=7,d=1,q=7

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

arima_model_train_7_7 = ARIMA(Train.price , order=(7,1,7)).fit()

pred_arima = arima_model_train_7_7.predict(start=Test.index[0],end=Test.index[-1])
ARIMA_MODEL_TRAIN_7_7=MAPE(pred_arima , Test.price)
ARIMA_MODEL_TRAIN_7_7

In [ ]:
fc=arima_model_train_7_7.forecast(143)
# Make as pandas series
fc_series = pd.Series(fc, index=Test.index)
#lower_series = pd.Series(conf[:, 0], index=test.index)
#upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(Train, label='training')
plt.plot(Test, label='actual')
plt.plot(fc_series, label='forecast')
#plt.fill_between(lower_series.index, upper_series, 
                 #color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

arima_model_train_4_1 = ARIMA(Train.price , order=(4,1,1)).fit()

pred_arima = arima_model_train_4_1.predict(start=Test.index[0],end=Test.index[-1])
ARIMA_MODEL_TRAIN_4_1=MAPE(pred_arima , Test.price)
ARIMA_MODEL_TRAIN_4_1

In [ ]:
fc=arima_model_train_4_1.forecast(143)
# Make as pandas series
fc_series = pd.Series(fc, index=Test.index)
#lower_series = pd.Series(conf[:, 0], index=test.index)
#upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
#plt.plot(Train, label='training')
plt.plot(Test, label='actual')
plt.plot(fc_series, label='forecast')
#plt.fill_between(lower_series.index, upper_series, 
                 #color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

# creating the SARIMA model

In [ ]:
sarima_model_train = sm.tsa.statespace.SARIMAX(Train['price'] , order=(1,1,1),seasonal_order=(1,1,1,7)).fit()
pred_sarima = sarima_model_train.predict(start=Test.index[0] , end=Test.index[-1])
SARIMA_MODEL=MAPE(pred_sarima , Test.price)
SARIMA_MODEL

In [ ]:
fc=sarima_model_train.forecast(143)
# Make as pandas series
fc_series = pd.Series(fc, index=Test.index)
#lower_series = pd.Series(conf[:, 0], index=test.index)
#upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(Train, label='training')
plt.plot(Test, label='actual')
plt.plot(fc_series, label='forecast')
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
fc=sarima_model_train.forecast(143)
# Make as pandas series
fc_series = pd.Series(fc, index=Test.index)
#lower_series = pd.Series(conf[:, 0], index=test.index)
#upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
#plt.plot(Train, label='training')
plt.plot(Test, label='actual')
plt.plot(fc_series, label='forecast')
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

# Creating the data frame for error percentage

In [ ]:
Error_percentage = {"MODEL":pd.Series(["Simple_exp_smoothing","Holt_exponential_model","Holt_winter_exp_smoothing_add_add","Holt_wilter_exp_smoothing_mul_add","Holt_wilter_exp_smoothing_add_mul",'Holt_wilter_exp_smoothing_mul_mul',"arima_model_train_1_1","arima_model_train_7_7","arima_model_train_31_31","arima_model_train_4_1","arima_model_train_1_4_1","SARIMA_Model"]),
        "MAPE_Values":pd.Series([simple_exp_model,double_exp_model,triple_exp_add_add,triple_exp_mul_add,triple_exp_add_mul,triple_exp_mul_mul,ARIMA_MODEL_TRAIN_1_1,ARIMA_MODEL_TRAIN_7_7,ARIMA_MODEL_TRAIN_31_31,ARIMA_MODEL_TRAIN_4_1,SARIMA_MODEL])}
table_MAPE=pd.DataFrame(Error_percentage)
table_MAPE
#table_MAPE.sort_values(['MAPE_Values'])

In [ ]:
#predicting the values test by using the ARIMA_MODEL_TRAIN 
#predicting the values for the next 30 days for test data
arima_model_train_31_31.forecast(143)

In [ ]:
gold_price

In [ ]:
#fitting the final model using the ARIMA model with p=7 , d=1 , q=7
FINAL_ARIMA_MODEL= ARIMA(gold_price['price'], order=(7,1,7)).fit()

In [ ]:
FINAL_ARIMA_MODEL.forecast(30) 

In [ ]:
Final_arima_model=FINAL_ARIMA_MODEL.forecast(30)
#make pandas as series
final_series = pd.Series(Final_arima_model)
#ploting
plt.figure(figsize=(12,5) , dpi=100)
plt.plot(gold_price , label='Actual data')
plt.plot(final_series,label='Forecasted data')
plt.title('Actual data vs Forecasted data')
plt.legend(loc='upper left' , fontsize=8)
plt.show()
#gold_price[['price', 'forecast']].plot(figsize=(12, 8))


In [ ]:
Final_arima_model=FINAL_ARIMA_MODEL.forecast(30)
#make pandas as series
final_series = pd.Series(Final_arima_model)
#ploting
plt.figure(figsize=(12,5) , dpi=100)
plt.plot(gold_price[1750:] , label='Actual data')
plt.plot(final_series,label='Forecasted data')
plt.title('Actual data vs Forecasted data')
plt.legend(loc='upper left' , fontsize=8)
plt.show()
#gold_price[['price', 'forecast']].plot(figsize=(12, 8))


# Grid Search For the Hyper Parameters

In [ ]:
# Import libraries

from numpy import sqrt
import warnings
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
# separate out a validation dataset
split_point = len(stationary_gold_price) - 143
dataset, validation = stationary_gold_price[0:split_point], stationary_gold_price[split_point:]
print('Dataset %d, Validation %d' % (len(dataset), len(validation)))
dataset.to_csv('dataset.csv', header=True)
validation.to_csv('validation.csv', header=False)

In [ ]:
# grid search ARIMA parameters for a time series

import warnings
from pandas import read_csv
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt


# evaluate an ARIMA model for a given order (p=2,d=4,q=3) and return RMSE
def evaluate_arima_model(X, arima_order):
# prepare training dataset
    X = X.astype('float32')
    train_size = int(len(X) * 0.80)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
# make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
# model_fit = model.fit(disp=0)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
# calculate out of sample error
    mape = MAPE(test, predictions)
    return mape

In [ ]:
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float('inf'), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    mape = evaluate_arima_model(train, order)
                    if mape < best_score:
                        best_score, best_cfg = mape, order
                    print('ARIMA%s MAPE=%.3f' % (order,mape))
                except Exception as e :
                    print(e)
                    continue
    print('Best ARIMA%s MAPE=%.3f' % (best_cfg, best_score))

In [ ]:
import statsmodels.api as sm

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
# load dataset
train = pd.read_csv('dataset.csv', index_col=0, parse_dates=True, squeeze=True)
# evaluate parameters
p_values = range(1, 5)
d_values = range(1, 5)
q_values = range(1, 5)
warnings.filterwarnings("ignore")
evaluate_models(train.values, p_values, d_values, q_values)